In [ ]:
# http://www.vworld.kr/dev/v4api.do (공간정보 오픈 플랫폼)
import pandas as pd
import folium
import urllib
import json
import re
import requests
from pyproj import Proj,transform
#스타벅스 csv파일을 연다
data1 = pd.read_csv('starbucks_loc.csv', encoding='utf-8')

# csv파일에 none값을 제거해준다.
data = data1.dropna()

#스타벅스 매장의 도로명 주소를 리스트 형식으로 받는다.
titledata = list(map(str,data['도로명전체주소'].tolist()))

# 오픈 api를 쉽게 이용하기 위해 html과 ID 를 미리 만들어 놓는다.
html = 'http://map.vworld.kr/search.do?category=juso&q='
ID = '&pageunit=10&output=json&pageindex=1&apiKey="본인 KEY 입력"'

# 지도에 매장의 위치를 보여주기 위해 folium을 사용한다.
m = folium.Map(location=[37.574665, 126.889048],zoom_start=15)

# 매장 수 만큼 for문을 반복한다.
for i in range(len(titledata)):
    #정규 표현식을 이용하여 도로명 주소의 필요없는 부분을 제거한다.
    try : Title = re.search(r'.*?[\(,]',titledata[i]).group(0)
    except Exception as e:
        continue
    Title = re.sub(r'[,\(]','',Title)

    #requests를 이용하여 오픈 api에서 원하는 값을 가져온다.
    r = requests.get(html+urllib.parse.quote(Title)+ID)
    #데이터를 json_data 변수에 저장한다.(json형식)
    json_data = r.json()
    #데이터에서 LIST 의 목록을 찾는다.
    try: a = json_data['LIST']
    except Exception as e:
        continue

    #LIST 목록에서 위도와 경도의 값을 추출한다.
    try : x = (a[0]['xpos'])
    except Exception as e:
        continue
    y = (a[0]['ypos'])

    # 추출한 위도와 경도를 이용하여 지도에 표시를 한다.
    folium.Marker(location = [y,x],popup='스타벅스',icon=folium.Icon(color='red',icon='star')).add_to(m)
    #과정이 진행되는지 확인하기 위해 위도와 경도를 출력한다.
    print(x,y)

# 표시가 다 끝났으면 map3.html에 지도를 저장한다.
m.save('map3.html')